<a href="https://colab.research.google.com/github/arpagon/podcast-coffeebreak/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Detect GPU

In [ ]:
!nvidia-smi

# Install CondaColab

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

# Check CondaColab it's ok

In [ ]:
import condacolab
condacolab.check()

# Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
podcast_audio_directory = '/content/drive/MyDrive/AI/podcast/audio'
podcast_text_directory = '/content/drive/MyDrive/AI/podcast/text'

In [ ]:
!mkdir -p /content/drive/MyDrive/AI/podcast/audio
!mkdir -p /content/drive/MyDrive/AI/podcast/text

# Clone Repo

In [ ]:
!git clone https://github.com/arpagon/podcast-coffeebreak.git

# Install my env

In [ ]:
!mamba env update -n base -f /content/podcast-coffeebreak/environment.yml

# Check youtube-dl Version

In [ ]:
!youtube-dl --version

# Download Podcast

In [ ]:
!bash /content/podcast-coffeebreak/utils/000_download_podcast.sh -u 'https://podcasts.google.com/feed/aHR0cDovL2ZlZWRzLmZlZWRidXJuZXIuY29tL1BvZGNhc3RDb2ZmZWVCcmVhaw?sa=X&ved=0CD4Q9sEGahcKEwjw1Zv4i8_6AhUAAAAAHQAAAAAQEw' -d /content/drive/MyDrive/AI/podcast/audio

In [ ]:
import os
import whisper
import json

model = whisper.load_model("medium")

for filename in os.listdir(podcast_audio_directory):
    audio_filename = os.path.join(podcast_audio_directory, filename)
    text_filename = os.path.join(podcast_text_directory, filename.split('.')[0] + '.json')
    if not os.path.exists(text_filename):
        if os.path.isfile(audio_filename):
            print('\r transcribing {0}'.format(audio_filename))
            result = model.transcribe(audio_filename)
            with open(text_filename, 'w',encoding='utf8') as text_filename_file:
                print('\r Writing {0}'.format(text_filename))
                json.dump(result, text_filename_file, indent=4)
